## OSM Preprocessing

In [2]:
!pip install osmnx google-cloud-storage rasterio rasterize > /dev/null 2>&1

In [4]:
import os
import geopandas as gpd
import osmnx as ox
import rasterio
from rasterio.features import rasterize
import matplotlib.pyplot as plt
import numpy as np

print("Kütüphaneler başarıyla yüklendi.")

ModuleNotFoundError: No module named 'osmnx'

In [ ]:
# In[2]:
# --- Konfigürasyon ve Gerekli Tanımlamalar ---

# Çalışılacak alan (Area of Interest - AOI)
place_name = "Hamburg, Germany"

# OSM verileri için kullanılacak etiketler
# Projenin ihtiyacına göre bu etiketler zenginleştirilebilir.
tags = {
    "building": True,
    "highway": True,
    "leisure": ["park", "garden", "playground"],
    "landuse": ["forest", "grass", "greenfield"],
    "natural": ["water", "wetland"]
}

# Çıktıların kaydedileceği klasör
PROCESSED_DIR = '../data/processed/'
os.makedirs(PROCESSED_DIR, exist_ok=True)
print(f"İşlenmiş veriler '{PROCESSED_DIR}' klasörüne kaydedilecek.")

# Referans Raster: Tüm çıktıların bu raster'ın boyut, projeksiyon (CRS) ve
# çözünürlüğüne hizalanması çok önemlidir. Bu, katmanların üst üste
# tam oturmasını sağlar. Genellikle 10m çözünürlüklü bir Sentinel-2 bandı kullanılır.
# LÜTFEN BU YOLU KENDİ PROJENİZDEKİ BİR SENTINEL-2 DOSYASININ YOLU İLE GÜNCELLEYİN!
REFERENCE_RASTER_PATH = '../data/raw/sentinel2/B04_10m.jp2' # Örnek: Kırmızı bant (10m)

if not os.path.exists(REFERENCE_RASTER_PATH):
    raise FileNotFoundError(f"Referans raster dosyası bulunamadı: {REFERENCE_RASTER_PATH}. Lütfen bu yolu projenizdeki 10m'lik bir Sentinel-2 bandı ile güncelleyin.")

print(f"Referans raster olarak '{REFERENCE_RASTER_PATH}' kullanılacak.")

In [ ]:
# In[3]:
# --- Adım 1: OSM'den Vektör Verilerini İndirme ---
# osmnx kütüphanesi ile Hamburg için tanımlanan etiketlere göre verileri indiriyoruz.
# Bu işlem internet hızınıza ve alanın büyüklüğüne göre biraz zaman alabilir.

print(f"'{place_name}' için OSM verileri indiriliyor...")
gdf = ox.geometries_from_place(place_name, tags)
print(f"Veri indirme tamamlandı. Toplam {len(gdf)} adet geometri bulundu.")

# İndirilen veriye hızlı bir bakış
print("İndirilen veri örneği (ilk 5 satır):")
print(gdf.head())

In [ ]:
# In[4]:
# --- Adım 2: Veri Katmanlarını Ayırma ve Kaydetme ---
# İndirilen tüm veriyi, modelde ayrı kanallar olarak kullanmak üzere
# (binalar, yollar, yeşil alanlar, su) katmanlarına ayıralım.

# Binalar
buildings = gdf[gdf['building'].notna()]
print(f"Toplam {len(buildings)} adet bina bulundu.")

# Yollar (Sadece çizgisel geometrileri alalım)
roads = gdf[gdf['highway'].notna() & (gdf.geom_type == 'LineString')]
print(f"Toplam {len(roads)} adet yol segmenti bulundu.")

# Yeşil Alanlar (Parklar, ormanlar, çimenlikler)
green_spaces = gdf[
    (gdf['leisure'].isin(['park', 'garden', 'playground'])) |
    (gdf['landuse'].isin(['forest', 'grass', 'greenfield']))
]
print(f"Toplam {len(green_spaces)} adet yeşil alan bulundu.")

# Su Kütleleri
water = gdf[gdf['natural'].isin(['water', 'wetland'])]
print(f"Toplam {len(water)} adet su kütlesi bulundu.")

In [ ]:
# In[5]:
# --- Adım 3: Vektör Katmanlarını Rasterleştirme (Rasterize) ---
# Bu adımda, ayırdığımız her bir vektör katmanını, referans Sentinel-2 görüntümüzle
# aynı boyut ve çözünürlükte bir raster (GeoTIFF) dosyasına dönüştüreceğiz. [cite: 151, 173]

def rasterize_and_save(gdf, reference_raster_path, output_path):
    """
    Bir GeoDataFrame'i referans raster'a göre rasterleştirir ve kaydeder.
    """
    # Referans raster'ı aç ve metaverilerini al
    with rasterio.open(reference_raster_path) as src:
        ref_meta = src.meta.copy()
        ref_transform = src.transform
        ref_shape = src.shape
        ref_crs = src.crs

    # Vektörün CRS'sini referans ile aynı yap
    if gdf.crs != ref_crs:
        gdf = gdf.to_crs(ref_crs)

    print(f"'{output_path}' için rasterleştirme başlıyor...")
    # Rasterize işlemi
    rasterized_array = rasterize(
        shapes=[(geom, 1) for geom in gdf.geometry], # Geometrilerin olduğu piksellere 1 yaz
        out_shape=ref_shape,
        transform=ref_transform,
        fill=0, # Geri kalan pikseller 0 olsun
        dtype=rasterio.uint8
    )

    # Yeni raster'ı kaydet
    ref_meta.update(dtype='uint8', count=1, compress='lzw')
    with rasterio.open(output_path, 'w', **ref_meta) as dst:
        dst.write(rasterized_array, 1)

    print(f"'{output_path}' başarıyla oluşturuldu.")
    return rasterized_array

# Her katman için rasterleştirme işlemini yapalım
building_mask = rasterize_and_save(buildings, REFERENCE_RASTER_PATH, os.path.join(PROCESSED_DIR, 'building_mask.tif'))
road_mask = rasterize_and_save(roads, REFERENCE_RASTER_PATH, os.path.join(PROCESSED_DIR, 'road_mask.tif'))
green_space_mask = rasterize_and_save(green_spaces, REFERENCE_RASTER_PATH, os.path.join(PROCESSED_DIR, 'green_space_mask.tif'))
water_mask = rasterize_and_save(water, REFERENCE_RASTER_PATH, os.path.join(PROCESSED_DIR, 'water_mask.tif'))

print("\nTüm OSM katmanları başarıyla rasterleştirildi ve kaydedildi.")

In [ ]:

# In[6]:
# --- Adım 4: Sonuçları Görselleştirme (Kontrol Amaçlı) ---
# Oluşturduğumuz raster maskelerden birini çizdirerek işlemin doğruluğunu kontrol edelim.

print("Bina maskesi (building_mask.tif) görselleştiriliyor...")

# Matplotlib ile görselleştirme
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.imshow(building_mask, cmap='viridis') # 1'ler sarı, 0'lar mor
ax.set_title('Rasterized Building Mask for Hamburg')
ax.set_axis_off() # Eksenleri kapat
plt.show()

In [ ]:
# In[7]:
# --- Özet ve Sonraki Adımlar ---
# Bu not defterinde, Hamburg için OSM verilerini indirip, modelimiz için gerekli olan
# 'binalar', 'yollar', 'yeşil alanlar' ve 'su' katmanlarını 10m çözünürlükte
# raster maskeler olarak hazırladık ve 'data/processed/' klasörüne kaydettik.
#
# Bir sonraki adım olan `03_data_processing.ipynb` not defterinde, bu maskeleri
# LST ve spektral indeksler (NDVI vb.) ile birleştirerek U-Net modelinin
# nihai girdisi olan çok kanallı tensörü oluşturacağız.